In [1]:
import nltk

from heapq import nlargest

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
from IPython.display import Markdown, display

In [2]:
nltk.download("punkt", quiet=True)
nltk.download("stopwords", quiet=True)

True

# Сумаризація тексту за допомогою `nltk`

1. [Суммаризация текста: подходы, алгоритмы, рекомендации и перспективы](https://habr.com/ru/articles/514540/)

Скористаємось алгоритмом екстрактивної сумаризації запропонованим в статті [1]

- Розбиття вхідного тексту на окремі речення 
- Переведення речення у цифрове представлення (вектор).
- Обчислення і збереження в матриці подібності подібності між векторами речень.
- Перетворення отриманої матриці на граф із реченнями у вигляді вершин і оцінками подібності у вигляді ребер для обчислення рангу речень.
- Вибір пропозицій з найвищою оцінкою для підсумкового резюме.

## Створюємо теку з українськими стоп-словами

In [3]:
import os
import requests

# Перевіряємо наявність папки corpora/stopwords у каталозі nltk_data
nltk_data_path = nltk.data.path[0]
stopwords_path = os.path.join(nltk_data_path, "corpora", "stopwords")
if not os.path.exists(stopwords_path):
    os.makedirs(stopwords_path)


# Завантажуємо стоп-слова для української мови та зберігаємо у файл
url = "https://raw.githubusercontent.com/skupriienko/Ukrainian-Stopwords/master/stopwords_ua.txt"
r = requests.get(url)

with open(os.path.join(stopwords_path, "ukrainian"), "wb") as f:
    f.write(r.content)


stop_words = set(stopwords.words("ukrainian"))


## Завантажуємо текст з файлу у змінну

In [4]:
with open("text_ua.txt", "r", encoding="utf-8") as file:
    text = file.read()

## Визначаємо функції

1. [Cosine Similarity and Cosine Distance](https://medium.com/geekculture/cosine-similarity-and-cosine-distance-48eed889a5c4)

In [5]:
def sentence_similarity(sent1, sent2, stopwords=None):
    """
    Функція вимірює схожість між двома реченнями. Використовується косинусна
    відстань між векторами, що представляють слова в реченнях. Вона також
    враховує стоп-слова.
    """
    if stopwords is None:
        stopwords = set()

    # Токенізуємо речення та видаляємо стоп-слова

    words1 = [
        word.lower()
        for word in word_tokenize(sent1)
        if word.isalnum() and word.lower() not in stopwords
    ]  # створюємо список слів (без стоп-слів) із першого речення
    words2 = [
        word.lower()
        for word in word_tokenize(sent2)
        if word.isalnum() and word.lower() not in stopwords
    ]  # створюємо список слів (без стоп-слів) із першого речення

    all_words = list(
        set(words1 + words2)
    )  # створюємо список унікальних слів з двох речень

    # Створюємо вектори типу компонентами яких є  одиниці та нулі
    # якщо слово є в речення, то ставимо 1, інакше 0
    # Отримуємо щось таке [1, 0, 0, 1, ...]
    vector1 = [1 if word in words1 else 0 for word in all_words]
    vector2 = [1 if word in words2 else 0 for word in all_words]

    # розраховуємо косинусну відстань між двома векторами

    return 1 - cosine_distance(vector1, vector2)


def build_similarity_matrix(sentences, stop_words):
    """
    Функція будує матрицю схожості між усіма парами речень у тексті.
    """
    similarity_matrix = np.zeros((len(sentences), len(sentences)))

    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i != j:
                similarity_matrix[i][j] = sentence_similarity(
                    sentences[i], sentences[j], stop_words
                )

    return similarity_matrix


def generate_summary(text, num_sentences=3, stop_words=stop_words):
    """
    Функція створює короткий зміст тексту.

    Алгоритм роботи:

    1. Читаємо речення із тексту і потім будуємо матрицю схожості.
    2. Розраховуємо суму схожості для кожного речення.
    3. Використовуємо nlargest для вибору топ N речень за порядком зменшення схожості.
    4. Об'єднуємо вибрані речення в один рядок.

    """
    summarize_text = []

    sentences = sent_tokenize(text)  # Токенізуємо текст

    sentence_similarity_matrix = build_similarity_matrix(sentences, stop_words)

    sentence_similarity_scores = np.array(
        [sum(row) for row in sentence_similarity_matrix]
    )

    # Використовуємо nlargest для вибору топ N речень за порядком зменшення схожості
    top_sentences_indices = nlargest(
        num_sentences,
        range(len(sentence_similarity_scores)),
        key=sentence_similarity_scores.__getitem__,
    )

    # Формуємо короткий зміст з вибраних речень
    for i in top_sentences_indices:
        summarize_text.append(sentences[i])

    return " ".join(summarize_text)

## Записуємо анотований текст до файлу

In [6]:
num_sentences = 3
stop_words = set(stopwords.words("ukrainian"))
summary = generate_summary(text, num_sentences, stop_words)
with open("summary_ua.txt", "w", encoding="utf-8") as file:
    file.write(summary)

# Висновки

Отже, ми маємо [вихідний текст](./text_ua.txt):


In [7]:
display(Markdown(text))

Орбітальний корабель "Діскавері", OV-103, вважається таким, що має право на внесення до Національного реєстру історичних місць (NRHP) в контексті Програми космічних човників США (1969-2011) за Критерієм А в галузі космічних досліджень і транспорту та за Критерієм С в галузі інженерії. Оскільки вона досягла значущості протягом останніх п'ятдесяти років, застосовується Розгляд критерію G. Згідно з Критерієм А, "Діскавері" є важливим як найстаріший з трьох збережених орбітальних кораблів, побудованих для програми "Спейс Шаттл" (SSP), найдовшої американської космічної програми на сьогоднішній день; він був третім з п'яти орбітальних кораблів, побудованих НАСА. На відміну від програм "Меркурій", "Джеміні" та "Аполлон", основна увага в SSP приділялася економічній ефективності та багаторазовому використанню, а в перспективі - створенню космічної станції. Включаючи свій перший політ (запущений 30 серпня 1984 року), "Діскавері" здійснив тридцять дев'ять польотів у космос, більше, ніж будь-який з інших чотирьох орбітальних апаратів; він також став першим орбітальним апаратом, який здійснив двадцять місій. Вона мала честь бути обраною для повернення до польотів після аварій "Челленджера" і "Колумбії". "Діскавері" став першим шатлом, який здійснив політ з модернізованими SRB після аварії "Челленджера", і першим шатлом, який здійснив політ з SSME Фази II і Блоку I. "Діскавері" також доставив на орбіту космічний телескоп "Габбл" і здійснив дві з п'яти місій з обслуговування обсерваторії. Вона виконала першу і останню місії Міністерства оборони США, а також першу несекретну місію, пов'язану з обороною. Крім того, "Діскавері" відіграв важливу роль у будівництві Міжнародної космічної станції (МКС); він здійснив тринадцять з тридцяти семи польотів на станцію за допомогою американського космічного човника. Вона була першим орбітальним кораблем, який пристикувався до МКС, і першим, який здійснив обмін екіпажем, що проживає на станції. За критерієм С "Діскавері" є важливим як інженерний подвиг. За словами Вейна Хейла, керівника польотів Космічного центру імені Джонсона, орбітальний корабель "Спейс Шаттл" являє собою "величезний технологічний стрибок від одноразових ракет і капсул до багаторазового, крилатого, гіперзвукового, вантажного космічного корабля". Хоча його базова конструкція наслідує конструкцію звичайного літака, він використовує передові матеріали, які мінімізують його вагу для перевезення вантажу і мають низький коефіцієнт теплового розширення, що забезпечує стабільну основу для матеріалів системи теплового захисту (TPS). На орбітальному кораблі "Спейс Шаттл" також була встановлена перша багаторазова система TPS; всі попередні космічні кораблі мали одноразовий абляційний теплозахисний екран. Серед інших визначних інженерних досягнень орбітального корабля - перша багаторазова орбітальна рухова установка і перша стійка до двох відмов інтегрована система авіоніки. За словами Хейла, "Спейс Шаттл" залишається "найбільшим, найшвидшим, найкрилатішим гіперзвуковим літаком в історії", який регулярно літає зі швидкістю, що у двадцять п'ять разів перевищує швидкість звуку.

та [його анотацію](./summary_ua.txt):

In [8]:
display(Markdown(summary))

Включаючи свій перший політ (запущений 30 серпня 1984 року), "Діскавері" здійснив тридцять дев'ять польотів у космос, більше, ніж будь-який з інших чотирьох орбітальних апаратів; він також став першим орбітальним апаратом, який здійснив двадцять місій. Крім того, "Діскавері" відіграв важливу роль у будівництві Міжнародної космічної станції (МКС); він здійснив тринадцять з тридцяти семи польотів на станцію за допомогою американського космічного човника. "Діскавері" став першим шатлом, який здійснив політ з модернізованими SRB після аварії "Челленджера", і першим шатлом, який здійснив політ з SSME Фази II і Блоку I.

In [9]:
print(f"Довжина вихідного тексту: {len(text)}")
print(f"Довжина анотації: {len(summary)}")


Довжина вихідного тексту: 3110
Довжина анотації: 621


Ту ж саму задачу я поставив `chatGPT`, ось його результат:

"Орбітальний корабель 'Діскавері' (OV-103) визнається історично значущим за Критеріями А та С в контексті Програми космічних човників США, як найстарший збережений орбітальний апарат, здійснивши 39 польотів, включаючи місії обслуговування телескопу 'Габбл' та важливий внесок у будівництво Міжнародної космічної станції, а також вперше використовуючи багаторазову систему теплового захисту та інші інновації в інженерії за останні п'ятдесят років."
